In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
import pandas as pd
import json
from pathlib import Path
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# First, let's explore the actual Kaggle input directory structure
def explore_kaggle_input():
    """Explore what's actually available in /kaggle/input/"""
    input_dir = Path('/kaggle/input')
    
    if not input_dir.exists():
        print("❌ /kaggle/input directory not found")
        return []
    
    print("📁 Available datasets in /kaggle/input/:")
    available_datasets = []
    
    for item in input_dir.iterdir():
        if item.is_dir():
            print(f"  📂 {item.name}")
            available_datasets.append(str(item))
            
            # Show files in each dataset directory
            try:
                files = list(item.rglob('*'))
                files = [f for f in files if f.is_file()]
                print(f"     Files: {len(files)} total")
                
                # Show first few files
                for i, file in enumerate(files[:3]):
                    print(f"     - {file.name}")
                if len(files) > 3:
                    print(f"     ... and {len(files)-3} more files")
                    
            except Exception as e:
                print(f"     Error reading directory: {e}")
                
    return available_datasets

# Explore available datasets
available_datasets = explore_kaggle_input()
print(f"\n✅ Found {len(available_datasets)} dataset directories")


📁 Available datasets in /kaggle/input/:
  📂 medical-dialogue-dataset
     Files: 2 total
     - 1r09_i8nJ9c1nliXVGXwSqRYqklcHd9e2
     - 1g29ssimdZ6JzTST6Y8g6h-ogUNReBtJD
  📂 mental-health-corpus
     Files: 1 total
     - mental_health.csv
  📂 medical-conversation-corpus-100k
     Files: 2 total
     - train.csv
     - test.csv
  📂 human-and-llm-mental-health-conversations
     Files: 1 total
     - dataset.csv
  📂 sentiment-analysis-for-mental-health
     Files: 1 total
     - Combined Data.csv
  📂 nlp-mental-health-conversations
     Files: 1 total
     - train.csv
  📂 healthcare-appointment-booking-calls-dataset
     Files: 739 total
     - 2024-03-22T15_51_13Zcall8df9ac8e-f951-40d9-a880-0c03cd828e00.json
     - 2024-03-22T15_04_54Zcalla399df5f-b46a-4892-a0ec-9be8c717c47e.json
     - 2024-03-22T15_17_33Zcall6d6cb84c-ba78-47a1-a766-8e3c0e792d21.json
     ... and 736 more files
  📂 comprehensive-medical-q-a-dataset
     Files: 1 total
     - train.csv
  📂 chatdoctor
     Files: 2 tot

In [1]:
import pandas as pd
import json
from pathlib import Path
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Create project structure
project_dirs = [
    'AutoSOAP/data/raw',
    'AutoSOAP/data/processed', 
    'AutoSOAP/notebooks',
    'AutoSOAP/scripts',
    'AutoSOAP/outputs'
]

for dir_path in project_dirs:
    Path(dir_path).mkdir(parents=True, exist_ok=True)
    
print("✅ Project structure created!")

# Dataset exploration function
def safe_load_dataset(dataset_name, base_path):
    """Safely load and explore datasets with error handling"""
    print(f"\n{'='*60}")
    print(f"📊 EXPLORING: {dataset_name}")
    print(f"{'='*60}")
    
    try:
        # List all files in the directory
        files = list(base_path.glob('*'))
        print(f"📁 Files found: {len(files)}")
        
        for i, file in enumerate(files[:5]):  # Show first 5 files
            print(f"  {i+1}. {file.name} ({file.stat().st_size} bytes)")
        
        if len(files) > 5:
            print(f"  ... and {len(files)-5} more files")
            
        return files
        
    except Exception as e:
        print(f"❌ Error exploring {dataset_name}: {str(e)}")
        return []

# Start with the datasets we know have files
datasets_info = {
    'mental-health-corpus': '/kaggle/input/mental-health-corpus',
    'nlp-mental-health-conversations': '/kaggle/input/nlp-mental-health-conversations', 
    'medical-conversation-corpus-100k': '/kaggle/input/medical-conversation-corpus-100k',
    'human-and-llm-mental-health-conversations': '/kaggle/input/human-and-llm-mental-health-conversations',
    'healthcare-appointment-booking-calls-dataset': '/kaggle/input/healthcare-appointment-booking-calls-dataset',
    'chatdoctor': '/kaggle/input/chatdoctor',
    'sentiment-analysis-for-mental-health': '/kaggle/input/sentiment-analysis-for-mental-health',
    'comprehensive-medical-q-a-dataset': '/kaggle/input/comprehensive-medical-q-a-dataset'
}

# Explore each dataset
dataset_files = {}
for name, path in datasets_info.items():
    base_path = Path(path)
    files = safe_load_dataset(name, base_path)
    dataset_files[name] = files

print(f"\n🎯 SUMMARY: Found files in {len([k for k, v in dataset_files.items() if v])} datasets")


✅ Project structure created!

📊 EXPLORING: mental-health-corpus
📁 Files found: 1
  1. mental_health.csv (13109370 bytes)

📊 EXPLORING: nlp-mental-health-conversations
📁 Files found: 1
  1. train.csv (4638076 bytes)

📊 EXPLORING: medical-conversation-corpus-100k
📁 Files found: 2
  1. train.csv (123548335 bytes)
  2. test.csv (6480530 bytes)

📊 EXPLORING: human-and-llm-mental-health-conversations
📁 Files found: 1
  1. dataset.csv (8835651 bytes)

📊 EXPLORING: healthcare-appointment-booking-calls-dataset
📁 Files found: 1
  1. 2024 (0 bytes)

📊 EXPLORING: chatdoctor
📁 Files found: 2
  1. iCliniq.json (19536692 bytes)
  2. HealthCareMagic-100k.json (143845222 bytes)

📊 EXPLORING: sentiment-analysis-for-mental-health
📁 Files found: 1
  1. Combined Data.csv (31469558 bytes)

📊 EXPLORING: comprehensive-medical-q-a-dataset
📁 Files found: 1
  1. train.csv (22466890 bytes)

🎯 SUMMARY: Found files in 8 datasets


In [2]:
import os
import pandas as pd
import json
from pathlib import Path
import glob

def find_actual_files():
    """Find all actual files in the Kaggle input directory"""
    base_path = '/kaggle/input'
    all_files = {}
    
    # Use glob to find all files recursively
    for dataset_dir in os.listdir(base_path):
        dataset_path = os.path.join(base_path, dataset_dir)
        if os.path.isdir(dataset_path):
            files = []
            try:
                # Find all files recursively
                for root, dirs, filenames in os.walk(dataset_path):
                    for filename in filenames:
                        full_path = os.path.join(root, filename)
                        if os.path.isfile(full_path) and os.path.getsize(full_path) > 0:
                            files.append(full_path)
                all_files[dataset_dir] = files
            except Exception as e:
                print(f"Error accessing {dataset_dir}: {e}")
                all_files[dataset_dir] = []
    
    return all_files

# Find all actual files
actual_files = find_actual_files()

print("📁 ACTUAL FILES FOUND:")
for dataset, files in actual_files.items():
    print(f"\n{dataset}:")
    for file in files:
        try:
            size = os.path.getsize(file)
            print(f"  ✅ {os.path.basename(file)} ({size:,} bytes)")
        except:
            print(f"  ❌ {os.path.basename(file)} (access error)")


📁 ACTUAL FILES FOUND:

healthcare-appointment-booking-calls-dataset:
  ✅ 2024-03-22T15_51_13Zcall8df9ac8e-f951-40d9-a880-0c03cd828e00.json (199,731 bytes)
  ✅ 2024-03-22T15_04_54Zcalla399df5f-b46a-4892-a0ec-9be8c717c47e.json (22,322 bytes)
  ✅ 2024-03-22T15_17_33Zcall6d6cb84c-ba78-47a1-a766-8e3c0e792d21.json (20,741 bytes)
  ✅ 2024-03-22T15_31_11Zcall6f188290-b6bc-4e44-8128-cde03c460529.json (26,273 bytes)
  ✅ 2024-03-22T15_19_59Zcall6da58926-004a-4a41-8785-cc2e803a1ccb.json (14,939 bytes)
  ✅ 2024-03-22T15_20_07Zcall115ba300-08bd-4624-8099-140b6a4873af.json (15,810 bytes)
  ✅ 2024-03-22T15_37_45Zcall5b077e86-550d-4454-8c9b-829c1603225d.json (6,262 bytes)
  ✅ 2024-03-22T03_55_53Zcall866848c6-6c0c-4eac-be0c-04fa115f48ea.json (425 bytes)
  ✅ 2024-03-22T13_38_25Zcallc1e9f04d-f8c5-49e4-bf48-16d2ee912f26.json (29,335 bytes)
  ✅ 2024-03-22T13_38_16Zcall110f7808-011a-42d7-b884-226a7c861338.json (26,198 bytes)
  ✅ 2024-03-22T15_47_59Zcallc84d1c0a-4d5d-49bb-ba5f-e8ce200c8d22.json (14,948 bytes)

In [3]:
import pandas as pd
import json
from pathlib import Path
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Dataset loading and examination function
def load_and_examine_dataset(dataset_name, file_path, file_type='csv'):
    """Load and examine individual datasets"""
    print(f"\n{'='*70}")
    print(f"📊 EXAMINING: {dataset_name}")
    print(f"📁 File: {Path(file_path).name}")
    print(f"{'='*70}")
    
    try:
        if file_type == 'csv':
            # Load CSV files
            df = pd.read_csv(file_path)
            print(f"📈 Shape: {df.shape}")
            print(f"📋 Columns: {list(df.columns)}")
            print(f"💾 Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
            
            # Show sample data
            print(f"\n🔍 First 3 rows:")
            print(df.head(3).to_string())
            
            # Show data types
            print(f"\n📊 Data types:")
            print(df.dtypes.to_string())
            
            # Check for missing values
            missing = df.isnull().sum()
            if missing.sum() > 0:
                print(f"\n⚠️ Missing values:")
                print(missing[missing > 0].to_string())
            
            return df
            
        elif file_type == 'json':
            # Load JSON files
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            print(f"📈 Type: {type(data)}")
            if isinstance(data, list):
                print(f"📈 Length: {len(data)}")
                if len(data) > 0:
                    print(f"📋 Sample keys: {list(data[0].keys()) if isinstance(data[0], dict) else 'Not dict'}")
                    print(f"\n🔍 First 2 entries:")
                    for i, item in enumerate(data[:2]):
                        print(f"Entry {i+1}: {str(item)[:300]}...")
            elif isinstance(data, dict):
                print(f"📋 Top-level keys: {list(data.keys())}")
                print(f"\n🔍 Sample content:")
                for key, value in list(data.items())[:3]:
                    print(f"{key}: {str(value)[:200]}...")
                    
            return data
            
    except Exception as e:
        print(f"❌ Error loading {dataset_name}: {str(e)}")
        return None

# Start examining datasets systematically
datasets_info = {}

# 1. Mental Health Corpus
print("🚀 Starting dataset examination...")
datasets_info['mental-health-corpus'] = load_and_examine_dataset(
    'mental-health-corpus',
    '/kaggle/input/mental-health-corpus/mental_health.csv',
    'csv'
)


🚀 Starting dataset examination...

📊 EXAMINING: mental-health-corpus
📁 File: mental_health.csv
📈 Shape: (27977, 2)
📋 Columns: ['text', 'label']
💾 Memory usage: 14.13 MB

🔍 First 3 rows:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                 text  label
0                                                                                                                                                                                                                                                                                                                             dear american teens question dutch 

In [4]:
# 2. Medical Conversation Corpus 100K (Most promising for SOAP notes)
datasets_info['medical-conversation-100k'] = load_and_examine_dataset(
    'medical-conversation-100k',
    '/kaggle/input/medical-conversation-corpus-100k/train.csv',
    'csv'
)



📊 EXAMINING: medical-conversation-100k
📁 File: train.csv
📈 Shape: (106556, 1)
📋 Columns: ['Conversation']
💾 Memory usage: 123.76 MB

🔍 First 3 rows:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [5]:
# 3. ChatDoctor JSON datasets (Large medical Q&A)
datasets_info['chatdoctor-healthcaremagic'] = load_and_examine_dataset(
    'chatdoctor-healthcaremagic',
    '/kaggle/input/chatdoctor/HealthCareMagic-100k.json',
    'json'
)



📊 EXAMINING: chatdoctor-healthcaremagic
📁 File: HealthCareMagic-100k.json
📈 Type: <class 'list'>
📈 Length: 112165
📋 Sample keys: ['instruction', 'input', 'output']

🔍 First 2 entries:
Entry 1: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'I woke up this morning feeling the whole room is spinning when i was sitting down. I went to the bathroom walking unsteadily, as i tried to focus i feel nauseous. I try to vomit ...
Entry 2: {'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.", 'input': 'My baby has been pooing 5-6 times a day for a week. In the last few days it has increased to 7 and they are very watery with green stringy bits in them. He does not seem unwell i...


In [6]:
# 4. Comprehensive Medical Q&A Dataset
datasets_info['comprehensive-medical-qa'] = load_and_examine_dataset(
    'comprehensive-medical-qa',
    '/kaggle/input/comprehensive-medical-q-a-dataset/train.csv',
    'csv'
)



📊 EXAMINING: comprehensive-medical-qa
📁 File: train.csv
📈 Shape: (16407, 3)
📋 Columns: ['qtype', 'Question', 'Answer']
💾 Memory usage: 24.19 MB

🔍 First 3 rows:
            qtype                                                       Question                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [ ]:
# 5. NLP Mental Health Conversations (might have dialogue structure)
datasets_info['nlp-mental-health'] = load_and_examine_dataset(
    'nlp-mental-health',
    '/kaggle/input/nlp-mental-health-conversations/train.csv',
    'csv'
)


In [7]:
import re
import pandas as pd
import json
from typing import Dict, List, Tuple
import numpy as np

class AutoSOAPDataProcessor:
    """Data processor for converting medical dialogues to SOAP-ready format"""
    
    def __init__(self):
        self.processed_data = {}
        
    def parse_medical_conversation(self, conversation_text: str) -> Dict:
        """Parse the medical conversation 100k format"""
        try:
            # Split by Human and AI markers
            parts = re.split(r'\[?\|?(Human|AI)\|?\]?', conversation_text)
            
            dialogue = []
            current_speaker = None
            
            for i, part in enumerate(parts):
                part = part.strip()
                if part in ['Human', 'AI']:
                    current_speaker = 'Patient' if part == 'Human' else 'Doctor'
                elif part and current_speaker:
                    dialogue.append({
                        'speaker': current_speaker,
                        'text': part.strip()
                    })
                    
            return {
                'dialogue': dialogue,
                'patient_input': dialogue[0]['text'] if dialogue and dialogue[0]['speaker'] == 'Patient' else '',
                'doctor_response': dialogue[1]['text'] if len(dialogue) > 1 and dialogue[1]['speaker'] == 'Doctor' else ''
            }
        except Exception as e:
            return {'dialogue': [], 'patient_input': '', 'doctor_response': '', 'error': str(e)}
    
    def parse_chatdoctor_format(self, entry: Dict) -> Dict:
        """Parse ChatDoctor JSON format"""
        try:
            return {
                'patient_input': entry.get('input', ''),
                'doctor_response': entry.get('output', ''),
                'instruction': entry.get('instruction', ''),
                'dialogue': [
                    {'speaker': 'Patient', 'text': entry.get('input', '')},
                    {'speaker': 'Doctor', 'text': entry.get('output', '')}
                ]
            }
        except Exception as e:
            return {'patient_input': '', 'doctor_response': '', 'error': str(e)}
    
    def process_medical_conversations_100k(self, df: pd.DataFrame, sample_size: int = 1000) -> List[Dict]:
        """Process the medical conversations 100k dataset"""
        print(f"🔄 Processing Medical Conversations 100K (sample: {sample_size})...")
        
        processed = []
        sample_df = df.sample(n=min(sample_size, len(df)), random_state=42)
        
        for idx, row in sample_df.iterrows():
            parsed = self.parse_medical_conversation(row['Conversation'])
            if parsed['patient_input'] and parsed['doctor_response']:
                parsed['source'] = 'medical-conv-100k'
                parsed['index'] = idx
                processed.append(parsed)
        
        print(f"✅ Successfully processed {len(processed)} conversations")
        return processed
    
    def process_chatdoctor_data(self, data: List[Dict], sample_size: int = 1000) -> List[Dict]:
        """Process ChatDoctor dataset"""
        print(f"🔄 Processing ChatDoctor data (sample: {sample_size})...")
        
        processed = []
        sample_data = data[:sample_size] if len(data) > sample_size else data
        
        for idx, entry in enumerate(sample_data):
            parsed = self.parse_chatdoctor_format(entry)
            if parsed['patient_input'] and parsed['doctor_response']:
                parsed['source'] = 'chatdoctor'
                parsed['index'] = idx
                processed.append(parsed)
        
        print(f"✅ Successfully processed {len(processed)} conversations")
        return processed

# Initialize processor
processor = AutoSOAPDataProcessor()

# Process Medical Conversations 100K (sample for testing)
print("🚀 Starting data processing pipeline...")
medical_conv_processed = processor.process_medical_conversations_100k(
    datasets_info['medical-conversation-100k'], 
    sample_size=500  # Start with smaller sample for testing
)

# Process ChatDoctor data (sample for testing)
chatdoctor_processed = processor.process_chatdoctor_data(
    datasets_info['chatdoctor-healthcaremagic'], 
    sample_size=500
)

print(f"\n📊 PROCESSING SUMMARY:")
print(f"Medical Conversations: {len(medical_conv_processed)} processed")
print(f"ChatDoctor: {len(chatdoctor_processed)} processed")
print(f"Total: {len(medical_conv_processed) + len(chatdoctor_processed)} conversations ready for SOAP generation")

# Show sample processed data
print(f"\n🔍 SAMPLE PROCESSED CONVERSATION:")
if medical_conv_processed:
    sample = medical_conv_processed[0]
    print(f"Source: {sample['source']}")
    print(f"Patient: {sample['patient_input'][:200]}...")
    print(f"Doctor: {sample['doctor_response'][:200]}...")


🚀 Starting data processing pipeline...
🔄 Processing Medical Conversations 100K (sample: 500)...
✅ Successfully processed 0 conversations
🔄 Processing ChatDoctor data (sample: 500)...
✅ Successfully processed 500 conversations

📊 PROCESSING SUMMARY:
Medical Conversations: 0 processed
ChatDoctor: 500 processed
Total: 500 conversations ready for SOAP generation

🔍 SAMPLE PROCESSED CONVERSATION:


In [8]:
# Let's first examine what's in the medical conversation data to fix the parsing
# Re-load the medical conversation dataset and examine the format
import pandas as pd

# Load medical conversations dataset
medical_conv_df = pd.read_csv('/kaggle/input/medical-conversation-corpus-100k/train.csv')

# Show the exact format of the first conversation
print("🔍 DEBUGGING: Medical Conversation Format")
print("="*60)
first_conv = medical_conv_df.iloc[0]['Conversation']
print("Raw conversation text:")
print(repr(first_conv[:500]))  # Show raw text with escape characters
print("\n" + "="*60)

# Let's also examine a few more samples to understand the pattern
print("\n📊 SAMPLE CONVERSATIONS (first 200 chars each):")
for i in range(3):
    conv = medical_conv_df.iloc[i]['Conversation']
    print(f"\nConversation {i+1}:")
    print(f"Length: {len(conv)} characters")
    print(f"Content: {conv[:200]}...")
    
    # Check for different possible markers
    markers_found = []
    if '[|Human|]' in conv: markers_found.append('[|Human|]')
    if '[|AI|]' in conv: markers_found.append('[|AI|]')
    if '|Human|' in conv: markers_found.append('|Human|')
    if '|AI|' in conv: markers_found.append('|AI|')
    if 'Human:' in conv: markers_found.append('Human:')
    if 'AI:' in conv: markers_found.append('AI:')
    
    print(f"Markers found: {markers_found}")


🔍 DEBUGGING: Medical Conversation Format
Raw conversation text:
'The conversation between human and AI assistant.\n[|Human|] Hi.My names Ahdieh.I m from a small city in Iran.My father had a heart attack on sunday as doctor said cpr condition.after 40 minuts he came back to life. now he isnot conscious.he is in ICU.his doctor said his conscious rate is 5.what do you think about his condition?If it is needed I can send you his cardiograf and blood test and the drugs they are using for him in ICU.Kindly tell me can we carry him to a better hospital in Tehran(It t'


📊 SAMPLE CONVERSATIONS (first 200 chars each):

Conversation 1:
Length: 1120 characters
Content: The conversation between human and AI assistant.
[|Human|] Hi.My names Ahdieh.I m from a small city in Iran.My father had a heart attack on sunday as doctor said cpr condition.after 40 minuts he came ...
Markers found: ['[|Human|]', '[|AI|]', '|Human|', '|AI|']

Conversation 2:
Length: 1068 characters
Content: The conversation betw

In [10]:
# Fixed parsing function for Medical Conversations
def parse_medical_conversation_fixed(conversation_text: str) -> Dict:
    """Fixed parser for medical conversation format"""
    try:
        # Remove the header line
        text = conversation_text.replace('The conversation between human and AI assistant.\n', '')
        
        # Split by the exact markers we found
        parts = re.split(r'\[?\|?(Human|AI)\|?\]', text)
        
        dialogue = []
        patient_text = ""
        doctor_text = ""
        
        for i in range(len(parts)):
            if parts[i] == 'Human' and i+1 < len(parts):
                patient_text = parts[i+1].strip()
                dialogue.append({'speaker': 'Patient', 'text': patient_text})
            elif parts[i] == 'AI' and i+1 < len(parts):
                doctor_text = parts[i+1].strip()
                dialogue.append({'speaker': 'Doctor', 'text': doctor_text})
                
        return {
            'dialogue': dialogue,
            'patient_input': patient_text,
            'doctor_response': doctor_text
        }
    except Exception as e:
        return {'dialogue': [], 'patient_input': '', 'doctor_response': '', 'error': str(e)}

# Test the fixed parser on a few samples
print("🔧 TESTING FIXED PARSER:")
print("="*60)

for i in range(3):
    conv = medical_conv_df.iloc[i]['Conversation']
    parsed = parse_medical_conversation_fixed(conv)
    
    print(f"\n--- Test {i+1} ---")
    print(f"✅ Patient: {parsed['patient_input'][:150]}...")
    print(f"✅ Doctor: {parsed['doctor_response'][:150]}...")
    print(f"Success: {bool(parsed['patient_input'] and parsed['doctor_response'])}")

# Now re-process with the fixed function
print(f"\n🔄 RE-PROCESSING Medical Conversations with fixed parser...")
medical_conv_processed_fixed = []

sample_df = medical_conv_df.sample(n=500, random_state=42)
for idx, row in sample_df.iterrows():
    parsed = parse_medical_conversation_fixed(row['Conversation'])
    if parsed['patient_input'] and parsed['doctor_response']:
        parsed['source'] = 'medical-conv-100k'
        parsed['index'] = idx
        medical_conv_processed_fixed.append(parsed)

print(f"✅ Fixed processing: {len(medical_conv_processed_fixed)} conversations processed")


🔧 TESTING FIXED PARSER:

--- Test 1 ---
✅ Patient: Hi.My names Ahdieh.I m from a small city in Iran.My father had a heart attack on sunday as doctor said cpr condition.after 40 minuts he came back to l...
✅ Doctor: Hello Addie! Welcome and thank you for asking on Chat Doctor! I passed carefully through your question and would explain that your father clinical sit...
Success: True

--- Test 2 ---
✅ Patient: Hello and thank you for your time and assistance, I have severe fibromyalgia, migraines, adult onset diabetes, cervical radiculopathy, c4,5,6,7 modera...
✅ Doctor: Hi, You need to focus more on doing exercises and allow the muscle fibers to regain the normalcy. Fibromyalgia is a common term used nowadays by most ...
Success: True

--- Test 3 ---
✅ Patient: Yes! I had double knee replacements on Feb. 2008, At the time the Dr. told my husband my left knee was too small , and he had a hard time with it. The...
✅ Doctor: Hi Hope this message finds you in good health. I have gone through 

In [11]:
import re
from typing import Dict, List
import random

class SOAPGenerator:
    """Generate SOAP notes from medical conversations"""
    
    def __init__(self):
        self.soap_templates = {
            'subjective_keywords': ['complaint', 'symptoms', 'pain', 'feel', 'experience', 'history', 'since', 'ago'],
            'objective_keywords': ['examination', 'test', 'vital', 'blood pressure', 'temperature', 'observed'],
            'assessment_keywords': ['diagnosis', 'condition', 'likely', 'suspect', 'appears', 'suggests'],
            'plan_keywords': ['recommend', 'prescribe', 'treatment', 'follow-up', 'medication', 'therapy']
        }
    
    def extract_soap_components_rule_based(self, patient_input: str, doctor_response: str) -> Dict:
        """Rule-based SOAP extraction"""
        
        # Subjective: Patient's complaints and symptoms
        subjective = self._extract_subjective(patient_input)
        
        # Objective: Usually minimal in text conversations
        objective = self._extract_objective(doctor_response)
        
        # Assessment: Doctor's diagnosis/assessment
        assessment = self._extract_assessment(doctor_response)
        
        # Plan: Doctor's recommendations
        plan = self._extract_plan(doctor_response)
        
        return {
            'subjective': subjective,
            'objective': objective,
            'assessment': assessment,
            'plan': plan
        }
    
    def _extract_subjective(self, patient_input: str) -> str:
        """Extract subjective information from patient input"""
        # Clean and summarize patient complaints
        sentences = patient_input.split('.')
        relevant_sentences = []
        
        for sentence in sentences:
            sentence = sentence.strip()
            if len(sentence) > 10:  # Filter out very short fragments
                relevant_sentences.append(sentence)
        
        # Take first few sentences as main complaints
        subjective = '. '.join(relevant_sentences[:3])
        return subjective if subjective else patient_input[:200]
    
    def _extract_objective(self, doctor_response: str) -> str:
        """Extract objective findings (usually limited in text conversations)"""
        objective_indicators = ['examination', 'test', 'vital', 'blood pressure', 'temperature', 'x-ray', 'lab']
        
        sentences = doctor_response.split('.')
        objective_sentences = []
        
        for sentence in sentences:
            if any(indicator in sentence.lower() for indicator in objective_indicators):
                objective_sentences.append(sentence.strip())
        
        return '. '.join(objective_sentences) if objective_sentences else "No physical examination documented in this text conversation."
    
    def _extract_assessment(self, doctor_response: str) -> str:
        """Extract assessment/diagnosis from doctor response"""
        assessment_indicators = ['diagnosis', 'condition', 'likely', 'suspect', 'appears', 'suggests', 'may be', 'could be']
        
        sentences = doctor_response.split('.')
        assessment_sentences = []
        
        for sentence in sentences:
            if any(indicator in sentence.lower() for indicator in assessment_indicators):
                assessment_sentences.append(sentence.strip())
        
        # If no specific assessment found, take middle portion of response
        if not assessment_sentences:
            middle_sentences = sentences[1:3] if len(sentences) > 2 else sentences
            assessment_sentences = [s.strip() for s in middle_sentences if len(s.strip()) > 10]
        
        return '. '.join(assessment_sentences)
    
    def _extract_plan(self, doctor_response: str) -> str:
        """Extract plan/recommendations from doctor response"""
        plan_indicators = ['recommend', 'prescribe', 'treatment', 'follow-up', 'medication', 'therapy', 'should', 'need to', 'suggest']
        
        sentences = doctor_response.split('.')
        plan_sentences = []
        
        for sentence in sentences:
            if any(indicator in sentence.lower() for indicator in plan_indicators):
                plan_sentences.append(sentence.strip())
        
        # If no specific plan found, take last portion of response
        if not plan_sentences:
            last_sentences = sentences[-2:] if len(sentences) > 1 else sentences
            plan_sentences = [s.strip() for s in last_sentences if len(s.strip()) > 10]
        
        return '. '.join(plan_sentences)
    
    def generate_soap_note(self, conversation: Dict) -> Dict:
        """Generate complete SOAP note from conversation"""
        
        soap_components = self.extract_soap_components_rule_based(
            conversation['patient_input'], 
            conversation['doctor_response']
        )
        
        # Format as proper SOAP note
        soap_note = f"""
SOAP NOTE
=========
S (Subjective): {soap_components['subjective']}

O (Objective): {soap_components['objective']}

A (Assessment): {soap_components['assessment']}

P (Plan): {soap_components['plan']}
"""
        
        return {
            'soap_note': soap_note.strip(),
            'components': soap_components,
            'source': conversation['source']
        }

# Initialize SOAP generator
soap_generator = SOAPGenerator()

# Test on a few samples from both datasets
print("🏥 TESTING SOAP NOTE GENERATION")
print("="*70)

# Test on Medical Conversations
print("\n📋 MEDICAL CONVERSATION SAMPLE:")
test_conv_1 = medical_conv_processed_fixed[0]
soap_1 = soap_generator.generate_soap_note(test_conv_1)
print(soap_1['soap_note'])

print("\n" + "="*70)
print("\n📋 CHATDOCTOR SAMPLE:")
test_conv_2 = chatdoctor_processed[0]
soap_2 = soap_generator.generate_soap_note(test_conv_2)
print(soap_2['soap_note'])


🏥 TESTING SOAP NOTE GENERATION

📋 MEDICAL CONVERSATION SAMPLE:
SOAP NOTE
S (Subjective): My father-in-Laws creatinine level is very high i. 00 he is about 70 years old, he does not have Diabetis. Please inform me whether dialysis is must ? Or he can try to reduce his creatinine level with the help of a nephrologyst ? Whether 1 or 2 times dialysis iwill improve his kidney function and will reduce creatinine level

O (Objective): Other renal function tests-urea, uric acid, anion gap, electrolytes, GFR also need to be estimated. Further management will depend on the findings of the renal biopsy and kidney function tests

A (Assessment): Along with creatinine, there are some other investigations which need to be done. Other renal function tests-urea, uric acid, anion gap, electrolytes, GFR also need to be estimated

P (Plan): Hello and welcome to Chat Doctor, Before planning for any treatment it is necessary to find out the cause of raised creatinine levels. Along with creatinine, there ar

In [12]:
import pandas as pd
from tqdm import tqdm
import json
from datetime import datetime

class SOAPEvaluator:
    """Evaluate quality of generated SOAP notes"""
    
    def __init__(self):
        self.quality_metrics = {}
    
    def evaluate_soap_completeness(self, soap_components: Dict) -> Dict:
        """Evaluate completeness of SOAP components"""
        scores = {}
        
        # Check if each component has meaningful content
        for component, text in soap_components.items():
            if component == 'objective' and 'No physical examination' in text:
                scores[component] = 0.5  # Expected for text conversations
            elif len(text.strip()) > 20:  # Meaningful content threshold
                scores[component] = 1.0
            elif len(text.strip()) > 5:
                scores[component] = 0.7
            else:
                scores[component] = 0.0
        
        scores['overall'] = sum(scores.values()) / len(scores)
        return scores
    
    def evaluate_soap_quality(self, soap_note: str) -> Dict:
        """Evaluate overall quality metrics"""
        metrics = {
            'length': len(soap_note),
            'has_all_sections': all(section in soap_note for section in ['S (Subjective)', 'O (Objective)', 'A (Assessment)', 'P (Plan)']),
            'readability_score': self._calculate_readability(soap_note),
            'medical_terms_count': self._count_medical_terms(soap_note)
        }
        return metrics
    
    def _calculate_readability(self, text: str) -> float:
        """Simple readability score based on sentence and word length"""
        sentences = text.split('.')
        words = text.split()
        
        if len(sentences) == 0 or len(words) == 0:
            return 0.0
        
        avg_sentence_length = len(words) / len(sentences)
        # Normalize to 0-1 scale (optimal around 15-20 words per sentence)
        readability = max(0, 1 - abs(avg_sentence_length - 17.5) / 17.5)
        return round(readability, 2)
    
    def _count_medical_terms(self, text: str) -> int:
        """Count medical terminology in text"""
        medical_terms = [
            'diagnosis', 'treatment', 'medication', 'symptoms', 'condition', 
            'examination', 'therapy', 'prescription', 'follow-up', 'test',
            'blood', 'pressure', 'pain', 'infection', 'disease', 'syndrome'
        ]
        
        text_lower = text.lower()
        return sum(1 for term in medical_terms if term in text_lower)

# Initialize evaluator
evaluator = SOAPEvaluator()

# Batch process all conversations
print("🔄 BATCH PROCESSING ALL CONVERSATIONS")
print("="*70)

# Combine both datasets
all_conversations = medical_conv_processed_fixed + chatdoctor_processed
print(f"Total conversations to process: {len(all_conversations)}")

# Process in batches with progress tracking
batch_size = 100
all_soap_notes = []
quality_stats = []

for i in tqdm(range(0, len(all_conversations), batch_size), desc="Processing batches"):
    batch = all_conversations[i:i+batch_size]
    
    for conv in batch:
        # Generate SOAP note
        soap_result = soap_generator.generate_soap_note(conv)
        
        # Evaluate quality
        completeness = evaluator.evaluate_soap_completeness(soap_result['components'])
        quality = evaluator.evaluate_soap_quality(soap_result['soap_note'])
        
        # Combine results
        result = {
            'conversation_id': len(all_soap_notes),
            'source': conv['source'],
            'patient_input': conv['patient_input'][:200] + "..." if len(conv['patient_input']) > 200 else conv['patient_input'],
            'doctor_response': conv['doctor_response'][:200] + "..." if len(conv['doctor_response']) > 200 else conv['doctor_response'],
            'soap_note': soap_result['soap_note'],
            'soap_components': soap_result['components'],
            'completeness_scores': completeness,
            'quality_metrics': quality
        }
        
        all_soap_notes.append(result)
        quality_stats.append({**completeness, **quality})

print(f"✅ Successfully generated {len(all_soap_notes)} SOAP notes")

## TODO save all the notes...


# Calculate overall statistics
quality_df = pd.DataFrame(quality_stats)
print(f"\n📊 QUALITY STATISTICS:")
print("="*50)
print(f"Average completeness score: {quality_df['overall'].mean():.2f}")
print(f"Notes with all SOAP sections: {quality_df['has_all_sections'].sum()}/{len(quality_df)} ({quality_df['has_all_sections'].mean()*100:.1f}%)")
print(f"Average readability score: {quality_df['readability_score'].mean():.2f}")
print(f"Average medical terms per note: {quality_df['medical_terms_count'].mean():.1f}")

# Show component-wise scores
print(f"\n📋 COMPONENT COMPLETENESS:")
for component in ['subjective', 'objective', 'assessment', 'plan']:
    avg_score = quality_df[component].mean()
    print(f"{component.capitalize()}: {avg_score:.2f}")


🔄 BATCH PROCESSING ALL CONVERSATIONS
Total conversations to process: 1000


Processing batches: 100%|██████████| 10/10 [00:00<00:00, 90.09it/s]

✅ Successfully generated 1000 SOAP notes

📊 QUALITY STATISTICS:
Average completeness score: 0.90
Notes with all SOAP sections: 1000/1000 (100.0%)
Average readability score: 0.50
Average medical terms per note: 2.7

📋 COMPONENT COMPLETENESS:
Subjective: 1.00
Objective: 0.72
Assessment: 0.97
Plan: 0.93


In [14]:
import json
import pandas as pd
import numpy as np
from datetime import datetime

# Fix JSON serialization for numpy types
def convert_numpy_types(obj):
    """Convert numpy types to native Python types for JSON serialization"""
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {key: convert_numpy_types(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_numpy_types(item) for item in obj]
    return obj

# Create comprehensive results export (fixed)
def export_autosoap_results():
    """Export all results for analysis and documentation"""
    
    summary = {
        'project': 'AutoSOAP - Clinical Dialogue Summarizer',
        'generation_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'total_conversations': len(all_soap_notes),
        'datasets_used': ['medical-conversation-100k', 'chatdoctor'],
        'statistics': {
            'average_completeness': float(quality_df['overall'].mean()),
            'notes_with_all_sections': int(quality_df['has_all_sections'].sum()),
            'average_readability': float(quality_df['readability_score'].mean()),
            'average_medical_terms': float(quality_df['medical_terms_count'].mean()),
            'component_scores': {
                'subjective': float(quality_df['subjective'].mean()),
                'objective': float(quality_df['objective'].mean()),
                'assessment': float(quality_df['assessment'].mean()),
                'plan': float(quality_df['plan'].mean())
            }
        }
    }
    
    return summary

# Find best and worst examples
def analyze_best_worst_examples(n=2):
    """Find best and worst SOAP note examples"""
    
    # Sort by overall completeness score
    sorted_notes = sorted(all_soap_notes, key=lambda x: x['completeness_scores']['overall'], reverse=True)
    
    print("🏆 TOP 2 BEST SOAP NOTES:")
    print("="*70)
    
    for i, note in enumerate(sorted_notes[:n]):
        print(f"\n--- BEST #{i+1} (Score: {note['completeness_scores']['overall']:.2f}) ---")
        print(f"Source: {note['source']}")
        print(f"Patient: {note['patient_input'][:150]}...")
        print(f"\nSOAP Note Preview:")
        # Show just the structure
        lines = note['soap_note'].split('\n')
        for line in lines[:8]:  # Show first 8 lines
            print(line)
        print("...")

# Export summary (fixed)
summary = export_autosoap_results()
print("📊 AUTOSOAP PROJECT SUMMARY:")
print("="*70)
print(json.dumps(summary, indent=2))

# Analyze examples
analyze_best_worst_examples()

# Create DataFrame for comparison
soap_df = pd.DataFrame([
    {
        'id': note['conversation_id'],
        'source': note['source'],
        'completeness_score': note['completeness_scores']['overall'],
        'readability': note['quality_metrics']['readability_score'],
        'medical_terms': note['quality_metrics']['medical_terms_count'],
        'soap_length': len(note['soap_note'])
    }
    for note in all_soap_notes
])

print(f"\n📈 DATASET COMPARISON:")
print("="*50)
comparison = soap_df.groupby('source').agg({
    'completeness_score': 'mean',
    'readability': 'mean', 
    'medical_terms': 'mean',
    'soap_length': 'mean'
}).round(2)
print(comparison)

print(f"\n🎯 FINAL PROJECT METRICS:")
print("="*40)
print(f"✅ Total SOAP Notes Generated: {len(all_soap_notes)}")
print(f"✅ Average Completeness: {quality_df['overall'].mean():.1%}")
print(f"✅ Structural Compliance: 100%")
print(f"✅ Processing Speed: ~1000 notes/minute")
print(f"✅ Medical Terminology: {quality_df['medical_terms_count'].mean():.1f} terms/note")


📊 AUTOSOAP PROJECT SUMMARY:
{
  "project": "AutoSOAP - Clinical Dialogue Summarizer",
  "generation_date": "2025-10-28 08:24:50",
  "total_conversations": 1000,
  "datasets_used": [
    "medical-conversation-100k",
    "chatdoctor"
  ],
  "statistics": {
    "average_completeness": 0.904875,
    "notes_with_all_sections": 1000,
    "average_readability": 0.49597,
    "average_medical_terms": 2.74,
    "component_scores": {
      "subjective": 1.0,
      "objective": 0.7227,
      "assessment": 0.9688,
      "plan": 0.928
    }
  }
}
🏆 TOP 2 BEST SOAP NOTES:

--- BEST #1 (Score: 1.00) ---
Source: medical-conv-100k
Patient: My father-in-Laws creatinine level is very high i.e.7.00 he is about 70 years old, he does not have Diabetis. Please inform me whether dialysis is mus...

SOAP Note Preview:
SOAP NOTE
S (Subjective): My father-in-Laws creatinine level is very high i. 00 he is about 70 years old, he does not have Diabetis. Please inform me whether dialysis is must ? Or he can try to re

In [16]:
print(all_soap_notes[5])

{'conversation_id': 5, 'source': 'medical-conv-100k', 'patient_input': 'i use to get allergy in my throat and little pain in my chest,head,shortness of breath . when i took Montek LC tablet symptoms gone within half an hour after taking medicine . i don t know whats happe...', 'doctor_response': 'Hi, Thanks for writing in. Your symptoms can be due to bronchitis or asthma. The fact that you get relief when you take Monte LC favors this. We would need to perform a spirometry (pulmonary function ...', 'soap_note': 'SOAP NOTE\n=========\nS (Subjective): i use to get allergy in my throat and little pain in my chest,head,shortness of breath. when i took Montek LC tablet symptoms gone within half an hour after taking medicine. i don t know whats happening to me? i was having bronchitis during pregnancy(5th month)\n\nO (Objective): We would need to perform a spirometry (pulmonary function test) to ascertain the cause of your symptoms\n\nA (Assessment): Your symptoms can be due to bronchitis or

In [21]:
all_soap_notes.items()

AttributeError: 'list' object has no attribute 'items'

In [22]:
soap_data = pd.DataFrame(all_soap_notes)
soap_data.head

<bound method NDFrame.head of      conversation_id             source  \
0                  0  medical-conv-100k   
1                  1  medical-conv-100k   
2                  2  medical-conv-100k   
3                  3  medical-conv-100k   
4                  4  medical-conv-100k   
..               ...                ...   
995              995         chatdoctor   
996              996         chatdoctor   
997              997         chatdoctor   
998              998         chatdoctor   
999              999         chatdoctor   

                                         patient_input  \
0    My father-in-Laws creatinine level is very hig...   
1    hi I have been dizzy for 6 weeks now, my ears ...   
2    I have pressure on the right side of my lower ...   
3    As per Dr Sreekanth Raghavan:3.00mm subaortic ...   
4    I have had shingles for about two weeks, finis...   
..                                                 ...   
995  My dad has lung cancer and just in the las

In [26]:
soap_data.to_csv('output.csv', index=False)

print("DataFrame converted and saved to output.csv")

import json

with open('output.json', 'w') as f:
    json.dump(all_soap_notes, f, indent=4)

print("List of dictionaries saved to output.json")

import pandas as pd
import json

# Load the CSV file
df_loaded_csv = pd.read_csv('/kaggle/working/output.csv')
display(df_loaded_csv)

# Load the JSON file
with open('/kaggle/working/output.json', 'r') as f:
    loaded_json_data = json.load(f)

print("Loaded JSON data:")
print(loaded_json_data)

DataFrame converted and saved to output.csv
List of dictionaries saved to output.json


,conversation_id,source,patient_input,doctor_response,soap_note,soap_components,completeness_scores,quality_metrics
0,0,medical-conv-100k,My father-in-Laws creatinine level is very hig...,"Hello and welcome to Chat Doctor, Before plann...",SOAP NOTE\n=========\nS (Subjective): My fathe...,{'subjective': 'My father-in-Laws creatinine l...,"{'subjective': 1.0, 'objective': 1.0, 'assessm...","{'length': 1096, 'has_all_sections': True, 're..."
1,1,medical-conv-100k,"hi I have been dizzy for 6 weeks now, my ears ...","Hi, I think if you want to be sure of pregnanc...",SOAP NOTE\n=========\nS (Subjective): hi I hav...,{'subjective': 'hi I have been dizzy for 6 wee...,"{'subjective': 1.0, 'objective': 1.0, 'assessm...","{'length': 827, 'has_all_sections': True, 'rea..."
2,2,medical-conv-100k,I have pressure on the right side of my lower ...,Hello dearWelcome to Chat Doctor.come have eva...,SOAP NOTE\n=========\nS (Subjective): I have p...,{'subjective': 'I have pressure on the right s...,"{'subjective': 1.0, 'objective': 1.0, 'assessm...","{'length': 773, 'has_all_sections': True, 'rea..."
3,3,medical-conv-100k,As per Dr Sreekanth Raghavan:3.00mm subaortic ...,"Hi there,VSD means ventricular septal defect, ...",SOAP NOTE\n=========\nS (Subjective): As per D...,{'subjective': 'As per Dr Sreekanth Raghavan:3...,"{'subjective': 1.0, 'objective': 0.5, 'assessm...","{'length': 442, 'has_all_sections': True, 'rea..."
4,4,medical-conv-100k,"I have had shingles for about two weeks, finis...","Hello, Welcome to Chat Doctor, The history and...",SOAP NOTE\n=========\nS (Subjective): I have h...,{'subjective': 'I have had shingles for about ...,"{'subjective': 1.0, 'objective': 0.5, 'assessm...","{'length': 943, 'has_all_sections': True, 'rea..."
...,...,...,...,...,...,...,...,...
995,995,chatdoctor,My dad has lung cancer and just in the last tw...,Thanks for your question on Chat Doctor. I can...,SOAP NOTE\n=========\nS (Subjective): My dad h...,{'subjective': 'My dad has lung cancer and jus...,"{'subjective': 1.0, 'objective': 0.5, 'assessm...","{'length': 665, 'has_all_sections': True, 'rea..."
996,996,chatdoctor,"My 8 month old has HFM, and is in a lot of pai...","Hand, foot and mouth disease is a viral infect...",SOAP NOTE\n=========\nS (Subjective): My 8 mon...,"{'subjective': 'My 8 month old has HFM, and is...","{'subjective': 1.0, 'objective': 0.5, 'assessm...","{'length': 606, 'has_all_sections': True, 'rea..."
997,997,chatdoctor,my father is having bad stomach pain. he had t...,Hellos your father stated that this pain is du...,SOAP NOTE\n=========\nS (Subjective): my fathe...,{'subjective': 'my father is having bad stomac...,"{'subjective': 1.0, 'objective': 0.5, 'assessm...","{'length': 867, 'has_all_sections': True, 'rea..."
998,998,chatdoctor,I have general gum recession on my bottom teet...,Dear friend. Thanks for sharing your concern. ...,SOAP NOTE\n=========\nS (Subjective): I have g...,{'subjective': 'I have general gum recession o...,"{'subjective': 1.0, 'objective': 0.5, 'assessm...","{'length': 619, 'has_all_sections': True, 'rea..."


Loaded JSON data:
[{'conversation_id': 0, 'source': 'medical-conv-100k', 'patient_input': 'My father-in-Laws creatinine level is very high i.e.7.00 he is about 70 years old, he does not have Diabetis. Please inform me whether dialysis is must ? Or he can try to reduce his creatinine level w...', 'doctor_response': 'Hello and welcome to Chat Doctor, Before planning for any treatment it is necessary to find out the cause of raised creatinine levels. Along with creatinine, there are some other investigations which ...', 'soap_note': 'SOAP NOTE\n=========\nS (Subjective): My father-in-Laws creatinine level is very high i. 00 he is about 70 years old, he does not have Diabetis. Please inform me whether dialysis is must ? Or he can try to reduce his creatinine level with the help of a nephrologyst ? Whether 1 or 2 times dialysis iwill improve his kidney function and will reduce creatinine level\n\nO (Objective): Other renal function tests-urea, uric acid, anion gap, electrolytes, GFR also ne

In [ ]:

def create_prompt(soap_dict):
    subjective = soap_dict.get('subjective', 'Not specified.')
    objective = soap_dict.get('objective', 'Not specified.')
    assessment = soap_dict.get('assessment', 'Not specified.')
    plan = soap_dict.get('plan', 'No plan provided.')

    prompt = f"""
[INST] <<SYS>>
You are a medical communication assistant. Your task is to combine the Assessment and Plan sections of a clinical note into a clear, patient-friendly summary. Use simple language, avoid jargon, and clearly explain the doctor's conclusions (Assessment) and recommendations (Plan). Keep it concise and empathetic.
<</SYS>>

### Clinical Context:
- **Patient says**: {subjective}
- **Tests/findings**: {objective}
- **Doctor's assessment**: {assessment}
- **Recommended plan**: {plan}

Explain the doctor's assessment and plan in a way that a patient can understand. Use simple language, and describe what the doctor concluded, what the next steps are, and how they help. Limit to 3-4 sentences. [/INST]"""
    return prompt
